# Final project scratchpad

In [1]:
import matplotlib.pyplot as plt
from skimage.io import imread
import os

from keras.models import Sequential, Model#, load_model
from keras.layers import GlobalAveragePooling2D, Dense, Flatten, Input, AveragePooling2D, Lambda
from keras_preprocessing.image import ImageDataGenerator
from keras.optimizers import Adam, RMSprop, SGD
from keras.callbacks import EarlyStopping
from keras.backend import l2_normalize, clear_session

#from keras import backend as K

import pandas as pd
from sklearn.model_selection import train_test_split

Using TensorFlow backend.


## 1. Create dataframe containing images and gender labels

In [2]:
# load meta data file containing names, gender and train/test flag
meta_df = pd.read_csv('/home/11332722/final_project/data/vggface2/identity_meta.csv', 
                      quotechar='"',skipinitialspace=True)

In [3]:
meta_df.head(5)

,Class_ID,Name,Sample_Num,Flag,Gender
0,n000001,14th_Dalai_Lama,424,0,m
1,n000002,A_Fine_Frenzy,315,1,f
2,n000003,A._A._Gill,205,1,m
3,n000004,A._J._Buckley,387,1,m
4,n000005,A._J._Pierzynski,229,1,m


In [4]:
# dictionary to replace gender m with 0, and f with 1
gender = {'m': 0,'f': 1} 

In [5]:
meta_df['Gender_Num'] = [gender[item] for item in meta_df.Gender] 

In [6]:
meta_df.head(5)

,Class_ID,Name,Sample_Num,Flag,Gender,Gender_Num
0,n000001,14th_Dalai_Lama,424,0,m,0
1,n000002,A_Fine_Frenzy,315,1,f,1
2,n000003,A._A._Gill,205,1,m,0
3,n000004,A._J._Buckley,387,1,m,0
4,n000005,A._J._Pierzynski,229,1,m,0


In [7]:
# create dataframe from test image files
import os                                                                                                             

def list_files(dir):
    r = []
    for root, dirs, files in os.walk(dir):
        for name in files:
            r.append(os.path.join(root, name))
    return r

lst_files = list_files('/home/11332722/Keras-VGGFace2-Testing/data/test')

test_files_df = pd.DataFrame(lst_files)
test_files_df.columns = ['Path']
test_files_df.head()

,Path
0,/home/11332722/Keras-VGGFace2-Testing/data/tes...
1,/home/11332722/Keras-VGGFace2-Testing/data/tes...
2,/home/11332722/Keras-VGGFace2-Testing/data/tes...
3,/home/11332722/Keras-VGGFace2-Testing/data/tes...
4,/home/11332722/Keras-VGGFace2-Testing/data/tes...


In [8]:
# check single row to make sure value is as expected
test_files_df['Path'][2]

'/home/11332722/Keras-VGGFace2-Testing/data/test/n006211/0168_01.jpg'

In [9]:
# add columns to test_files_df
new = test_files_df["Path"].str.split("/", n = 7, expand = True) 

In [10]:
# add File_Name to test_files_df
test_files_df["File_Name"]= new[7] 
  
# add Class_ID to test_files_df
test_files_df["Class_ID"]= new[6] 

In [11]:
#check df
test_files_df.head()

,Path,File_Name,Class_ID
0,/home/11332722/Keras-VGGFace2-Testing/data/tes...,None,test_list.txt
1,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0089_01.jpg,n006211
2,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0168_01.jpg,n006211
3,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0213_01.jpg,n006211
4,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0010_01.jpg,n006211


In [12]:
# merge test_files_df with  meta_df
test_files_df = test_files_df.merge(meta_df,how='inner',on='Class_ID')

In [13]:
test_files_df.head()

,Path,File_Name,Class_ID,Name,Sample_Num,Flag,Gender,Gender_Num
0,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0089_01.jpg,n006211,Mohammad_Ali,249,0,m,0
1,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0168_01.jpg,n006211,Mohammad_Ali,249,0,m,0
2,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0213_01.jpg,n006211,Mohammad_Ali,249,0,m,0
3,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0010_01.jpg,n006211,Mohammad_Ali,249,0,m,0
4,/home/11332722/Keras-VGGFace2-Testing/data/tes...,0115_01.jpg,n006211,Mohammad_Ali,249,0,m,0


1) import code provided by VGGFace2 team @ https://drive.google.com/file/d/1AHVpuB24lKAqNyRRjhX7ABlEor6ByZlS/view
2) 


## 2. Create train and test datasets

In [14]:
# Create train and test sets 

test_files_df = test_files_df.sample(n=50000)

train_df , test_df = train_test_split(test_files_df, test_size = 0.2, shuffle = True , random_state = 44,
                                              stratify = test_files_df['Gender_Num']) # trmobr if problems

In [15]:
# Reset df indexes because the flow_from_dataframe expects a 0 index
train_df.reset_index(inplace=True)
test_df.reset_index(inplace=True)

## 3. Import ResNet50 model trained on VGGFace2 (from WeidiXie gihub)

In [16]:
# import code provided by VGGFace2 team @ https://drive.google.com/file/d/1AHVpuB24lKAqNyRRjhX7ABlEor6ByZlS/view
# this link came from https://github.com/WeidiXie/Keras-VGGFace2-ResNet50 (in the Model section)
os.chdir(r"/jupyteruser/11332722/vggface2_Keras/tool/")
import toolkits

os.chdir(r"/jupyteruser/11332722/vggface2_Keras/src/")
import config
import resnet
import utils
import model # commented out import new_layers

In [17]:
# commented out toolkits.initialize_GPU(args) but still didn't work
#%run "/jupyteruser/11332722/vggface2_Keras/src/predict.py"

In [18]:
#clear_session()
original_model = model.Vggface2_ResNet50(mode='train')

In [19]:
original_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
base_input (InputLayer)         (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        base_input[0][0]                 
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [20]:
# load pre-trained weights
weights_file = '/home/11332722/vggface2_Keras/model/resnet50_softmax_dim512/weights.h5'
original_model.load_weights(weights_file, by_name=False) # by_name = True for fine tuning or transfer learning (if model = 'eval')

## 3. Modify model via Feature Extraction and Fine Tuning (former is a must, second is optional but worth trying)

A few useful references:<br/>

**Replacing layers**<br/>

https://towardsdatascience.com/transfer-learning-from-pre-trained-models-f2393f124751<br/>

https://www.tensorflow.org/alpha/tutorials/images/transfer_learning<br/>
* collab notebook at: 
https://colab.research.google.com/github/tensorflow/docs/blob/master/site/en/r2/tutorials/images/transfer_learning.ipynb<br/>

https://www.learnopencv.com/keras-tutorial-transfer-learning-using-pre-trained-models/<br/>

https://github.com/tensorflow/tensorflow/issues/22479<br/>

**Similar project**<br/>

http://gigantes.ii.uam.es/fierrez/files/2018_CIARP_BiasFace_Acien.pdf

### 3.1 Feature Extraction (i.e. freeze convolutional base and add a new classification head)

### Approach 1 (create a sequential model)

In [21]:
# create new base model (includes all original layers up to classification)
layer_name = 'activation_49'
conv_base_model= Model(inputs=original_model.input, 
                       outputs=original_model.get_layer(layer_name).output,
                       name = 'conv_base_model')

In [22]:
conv_base_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
base_input (InputLayer)         (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        base_input[0][0]                 
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [23]:
conv_base_model.trainable = False


In [24]:
# check last layer of conv_base_model
conv_base_model.layers[-1].output.shape

TensorShape([Dimension(None), Dimension(7), Dimension(7), Dimension(2048)])

In [25]:
# global_average_layer = GlobalAveragePooling2D(name='avg_pool')
# feature_batch_average = global_average_layer(conv_base_model.layers[-1].output)
# #feature_batch_average = Flatten()(feature_batch_average)
# feature_batch_average = Dense(512, activation='relu', name='dim_proj')(feature_batch_average)
# #feature_batch_average = Flatten()(feature_batch_average)
# #feature_batch_average = Lambda(lambda x: l2_normalize(x, 1))(feature_batch_average)

# print(feature_batch_average)

In [26]:
global_average_layer = GlobalAveragePooling2D(name = 'global_average_layer')
feature_batch_average = global_average_layer(conv_base_model.layers[-1].output)
print(feature_batch_average)

Tensor("global_average_layer/Mean:0", shape=(?, 2048), dtype=float32)


In [27]:
prediction_layer = Dense(1,activation = 'sigmoid', name = 'prediction_layer')
prediction_batch = prediction_layer(feature_batch_average)
print(prediction_batch.shape)

(?, 1)


In [28]:
conv_base_model.trainable = False

model = Sequential([
  conv_base_model,
  global_average_layer,
  prediction_layer
])

In [29]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv_base_model (Model)      (None, 7, 7, 2048)        23561152  
_________________________________________________________________
global_average_layer (Global (None, 2048)              0         
_________________________________________________________________
prediction_layer (Dense)     (None, 1)                 2049      
Total params: 23,563,201
Trainable params: 2,049
Non-trainable params: 23,561,152
_________________________________________________________________


In [30]:
len(model.trainable_weights)

2

In [31]:
base_learning_rate = 0.0001
opt = Adam()
#opt = SGD()
# opt = RMSprop(lr=base_learning_rate)

model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [32]:
batch_size = 32
height = 224
width = 224

datagen = ImageDataGenerator(rescale=1./255)#, preprocessing_function=resize_image)

train_generator = datagen.flow_from_dataframe(    
        dataframe=train_df,
        x_col='Path',
        directory=None,
        y_col = 'Gender',
        target_size=(height, width),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='binary')

test_generator = datagen.flow_from_dataframe(    
        dataframe=test_df,
        x_col='Path',
        directory=None,
        y_col = 'Gender',
        target_size=(height, width),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='binary')

Found 40000 validated image filenames belonging to 2 classes.
Found 10000 validated image filenames belonging to 2 classes.


In [33]:
train_len = len(train_df)
test_len = len(test_df)
print(train_len)
print(test_len)

40000
10000


In [34]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')


In [35]:
history = model.fit_generator(train_generator,
                  steps_per_epoch= train_len // batch_size,
                  #shuffle=True,
                  epochs=10,
                  validation_data=test_generator,
                  validation_steps= test_len // batch_size,
                  callbacks=[early_stopping]
                           )

Epoch 1/10
1250/1250 [==============================] - 361s 289ms/step - loss: 0.4604 - acc: 0.8958 - val_loss: 2.0314 - val_acc: 0.6009
Epoch 2/10
1250/1250 [==============================] - 357s 286ms/step - loss: 0.4038 - acc: 0.9187 - val_loss: 2.2295 - val_acc: 0.6014
Epoch 3/10
1250/1250 [==============================] - 356s 285ms/step - loss: 0.3754 - acc: 0.9265 - val_loss: 1.8963 - val_acc: 0.6008
Epoch 4/10
1250/1250 [==============================] - 356s 285ms/step - loss: 0.3764 - acc: 0.9278 - val_loss: 2.1464 - val_acc: 0.6005
Epoch 5/10
1250/1250 [==============================] - 356s 285ms/step - loss: 0.3675 - acc: 0.9306 - val_loss: 2.2252 - val_acc: 0.6020
Epoch 6/10
1250/1250 [==============================] - 356s 284ms/step - loss: 0.3658 - acc: 0.9324 - val_loss: 2.3047 - val_acc: 0.6009
Epoch 7/10
 737/1250 [================>.............] - ETA: 1:54 - loss: 0.3576 - acc: 0.9361

KeyboardInterrupt: 

### Approach 2 (ok, but approach 1 feels cleaner)

In [ ]:
#x = GlobalAveragePooling2D()(new_base_model.layers[-1].output)

In [ ]:
#o = Activation('sigmoid', name='loss')(x)

In [ ]:
#new_model = Model(inputs=new_base_model.layers[0].input, outputs=[o])

In [ ]:
#new_model.summary()

In [ ]:
#input_dim=(224, 224, 3)

#inputs = Input(shape=input_dim, name='base_input')

In [36]:
x = AveragePooling2D((7, 7), name='avg_pool')(conv_base_model.layers[-1].output)
x = Flatten()(x)
x = Dense(512, activation='relu', name='dim_proj')(x)
x = Dense(512, activation='relu', name='dim_proj2')(x) # test
#x = Lambda(lambda x: l2_normalize(x, 1))(x)
o = Dense(1,activation='sigmoid', name = 'prediction_layer')(x)

new_model = Model(inputs=conv_base_model.layers[0].input, outputs=[o])

In [37]:
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
base_input (InputLayer)         (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        base_input[0][0]                 
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [38]:
new_model.trainable = True

In [40]:
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
base_input (InputLayer)         (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        base_input[0][0]                 
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [41]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(new_model.layers))

Number of layers in the base model:  178


In [42]:
# Fine tune from this layer onwards
fine_tune_at = 140

# Freeze all the layers before the `fine_tune_at` layer
for layer in new_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [43]:
new_model.summary()

__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
base_input (InputLayer)         (None, 224, 224, 3)  0                                            
__________________________________________________________________________________________________
conv1/7x7_s2 (Conv2D)           (None, 112, 112, 64) 9408        base_input[0][0]                 
__________________________________________________________________________________________________
conv1/7x7_s2/bn (BatchNormaliza (None, 112, 112, 64) 256         conv1/7x7_s2[0][0]               
__________________________________________________________________________________________________
activation_1 (Activation)       (None, 112, 112, 64) 0           conv1/7x7_s2/bn[0][0]            
__________________________________________________________________________________________________
max_poolin

In [44]:
base_learning_rate = 0.0001
opt = Adam()
# opt = RMSprop(lr=base_learning_rate)

new_model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [45]:
early_stopping = EarlyStopping(monitor='val_loss', min_delta=0, patience=5, verbose=1, mode='auto')


In [46]:
history = new_model.fit_generator(train_generator,
                  steps_per_epoch= train_len // batch_size,
                  #shuffle=True,
                  epochs=10,
                  validation_data=test_generator,
                  validation_steps= test_len // batch_size,
                  callbacks=[early_stopping]
                           )

Epoch 1/10
1250/1250 [==============================] - 478s 383ms/step - loss: 0.3484 - acc: 0.9487 - val_loss: 0.8655 - val_acc: 0.5992
Epoch 2/10
1250/1250 [==============================] - 471s 377ms/step - loss: 0.3083 - acc: 0.9617 - val_loss: 0.9107 - val_acc: 0.3978
Epoch 3/10
1250/1250 [==============================] - 472s 377ms/step - loss: 0.2976 - acc: 0.9662 - val_loss: 0.8863 - val_acc: 0.6016
Epoch 4/10
 587/1250 [=============>................] - ETA: 3:29 - loss: 0.2972 - acc: 0.9679

KeyboardInterrupt: 

In [ ]:
# Ignore below...

In [ ]:
new_model.trainable = True

In [ ]:
# Fine tune from this layer onwards
fine_tune_at = 150

# Freeze all the layers before the `fine_tune_at` layer
for layer in new_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
base_learning_rate = 0.0001
opt = Adam()
# opt = RMSprop(lr=base_learning_rate)

new_model.compile(optimizer=opt,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = new_model.fit_generator(train_generator,
                  steps_per_epoch= train_len // batch_size,
                  #shuffle=True,
                  epochs=10,
                  validation_data=test_generator,
                  validation_steps= test_len // batch_size,
                  callbacks=[early_stopping]
                           )

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
base_learning_rate = 0.0001
new_model.compile(optimizer=RMSprop(lr=base_learning_rate),
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [ ]:
history = new_model.fit_generator(train_generator,
                  steps_per_epoch= train_len // batch_size,
                  shuffle=True,
                  epochs=1,
                  validation_data=test_generator,
                  validation_steps= test_len // batch_size,
                  callbacks=[early_stopping]
                           )

In [ ]:
# generator
# to fix issue where flow_from_dataframe does not accept "directory=None"
# from https://github.com/keras-team/keras-preprocessing/issues/116
#pip install keras
#pip uninstall keras-preprocessing -y
#pip install git+https://github.com/keras-team/keras-preprocessing.git

batch_size = 32
height = 224
width = 224

datagen = ImageDataGenerator(rescale=1./255)#, preprocessing_function=resize_image)

test_generator = datagen.flow_from_dataframe(
        dataframe=test_files_df,
        directory=None,
        x_col='Path',
        target_size=(height, width),
        color_mode='rgb',
        batch_size=batch_size,
        class_mode='input')

In [ ]:
num_train, num_val, num_test = (
  metadata.splits['train'].num_examples*weight/10
  for weight in SPLIT_WEIGHTS
)

In [ ]:
initial_epochs = 10
steps_per_epoch = round(num_train)//BATCH_SIZE
validation_steps = 20

loss0,accuracy0 = model.evaluate(validation_batches, steps = validation_steps)

In [ ]:
print("initial loss: {:.2f}".format(loss0))
print("initial accuracy: {:.2f}".format(accuracy0))

In [ ]:
history = model.fit(train_batches,
                    epochs=initial_epochs,
                    validation_data=validation_batches)

In [ ]:
# check learning curves
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']

loss = history.history['loss']
val_loss = history.history['val_loss']

plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.legend(loc='lower right')
plt.ylabel('Accuracy')
plt.ylim([min(plt.ylim()),1])
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.legend(loc='upper right')
plt.ylabel('Cross Entropy')
plt.ylim([0,1.0])
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()

In [ ]:
preds = original_model.predict_generator(test_generator, steps=10,verbose=0)

In [ ]:
some_list = []
for pred in preds:
    #print(pred)
    some_list.append(np.argmax(pred))

np.unique(some_list)

In [ ]:
preds[1]

In [ ]:
if '':
    print('1')

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

In [ ]:
img_path = '/jupyteruser/11332722/Keras-VGGFace2-Testing/data/test/n003001/0001_01.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])

In [ ]:
# create test input to see if model runs
lst = []
lst = np.zeros(shape=(1,224, 224, 3))
lst.shape

In [ ]:
# load test image and take a look
test_file = '/jupyteruser/11332722/Keras-VGGFace2-Testing/data/test/n000001/0001_01.jpg'
test_img = imread(test_file,as_gray=False)
print(test_img.shape)
plt.imshow(test_img)

In [ ]:
# add image to test batch
np.append(lst,np.array(load_data(path=test_file,shape=(224,224,3))))
lst.shape

In [ ]:
plt.imshow(load_data(test_file,(224,224,3),mode='eval'))

In [ ]:
# run model
test = original_model.predict(lst)
test.shape

In [ ]:
x  = pd.DataFrame(test)
x[x.idxmax(1)]



### 2.2 Fine tuning (i.e. unfreeze based model and set set bottom layers to be un-trainable)

In [ ]:
conv_base_model.trainable = True

In [ ]:
# Let's take a look to see how many layers are in the base model
print("Number of layers in the base model: ", len(base_model.layers))

# Fine tune from this layer onwards
fine_tune_at = 100

# Freeze all the layers before the `fine_tune_at` layer
for layer in base_model.layers[:fine_tune_at]:
  layer.trainable =  False

In [ ]:
# Compile the model
model.compile(loss='binary_crossentropy',
              optimizer = tf.keras.optimizers.RMSprop(lr=base_learning_rate/10),
              metrics=['accuracy'])

In [ ]:
fine_tune_epochs = 10
total_epochs =  initial_epochs + fine_tune_epochs

history_fine = model.fit(train_batches,
                         epochs=total_epochs,
                         initial_epoch = initial_epochs,
                         validation_data=validation_batches)

In [ ]:
plt.figure(figsize=(8, 8))
plt.subplot(2, 1, 1)
plt.plot(acc, label='Training Accuracy')
plt.plot(val_acc, label='Validation Accuracy')
plt.ylim([0.8, 1])
plt.plot([initial_epochs-1,initial_epochs-1],
          plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='lower right')
plt.title('Training and Validation Accuracy')

plt.subplot(2, 1, 2)
plt.plot(loss, label='Training Loss')
plt.plot(val_loss, label='Validation Loss')
plt.ylim([0, 1.0])
plt.plot([initial_epochs-1,initial_epochs-1],
         plt.ylim(), label='Start Fine Tuning')
plt.legend(loc='upper right')
plt.title('Training and Validation Loss')
plt.xlabel('epoch')
plt.show()